# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df = pd.DataFrame(data['Quantity'].groupby([data['CustomerID'], data['ProductName']]).sum())
df.head(10)

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
           Beef - Chuck, Boneless                1
           Beef - Prime Rib Aaa                  1
           Beer - Original Organic Lager         1
           Beer - Rickards Red                   1
           Black Currants                        1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
pivot = pd.pivot_table(df,index=["ProductName"], columns=["CustomerID"], values = ["Quantity"], fill_value=0)
pivot.head(10)

Quantity                                \
CustomerID                           33    200   264   356   412   464     
ProductName                                                                
Anchovy Paste - 56 G Tube                0     0     0     0     0     0   
Appetizer - Mini Egg Roll, Shrimp        0     0     0     0     0     0   
Appetizer - Mushroom Tart                0     0     0     0     0     0   
Appetizer - Sausage Rolls                0     0     0     0     0     0   
Apricots - Dried                         1     0     0     0     1     0   
Apricots - Halves                        0     0     1     0     0     0   
Apricots Fresh                           0     0     1     0     0     1   
Arizona - Green Tea                      0     0     0     0     0     0   
Artichokes - Jerusalem                   0     0     0     0     0     0   
Assorted Desserts                        1     0     0     0     0     0   

                                                           ...              \
CustomerID                        477   639   649   669    ... 97697 97753   
ProductName                                                ...               
Anchovy Paste - 56 G Tube             0     1     0     0  ...     0    25   
Appetizer - Mini Egg Roll, Shrimp     0     0     0     0  ...    25    25   
Appetizer - Mushroom Tart             0     1     0     0  ...    25     0   
Appetizer - Sausage Rolls             0     0     0     0  ...     0     0   
Apricots - Dried                      0     0     0     0  ...     0    25   
Apricots - Halves                     0     0     0     0  ...     0     0   
Apricots Fresh                        0     0     0     0  ...     0     0   
Arizona - Green Tea                   0     0     1     0  ...     0     0   
Artichokes - Jerusalem                0     0     0     0  ...     0     0   
Assorted Desserts                     0     1     0     0  ...     0     0   

                                                                             \
CustomerID                        97769 97793 97900 97928 98069 98159 98185   
ProductName                                                                   
Anchovy Paste - 56 G Tube             0     0     0     0     0     0     0   
Appetizer - Mini Egg Roll, Shrimp     0     0     0     0     0     0     0   
Appetizer - Mushroom Tart             0     0     0     0     0     0    25   
Appetizer - Sausage Rolls             0     0    25    25    25     0    25   
Apricots - Dried                      0     0     0     0     0     0     0   
Apricots - Halves                     0     0    25    50    25     0    25   
Apricots Fresh                        0     0     0     0     0     0     0   
Arizona - Green Tea                   0     0     0    25     0     0     0   
Artichokes - Jerusalem                0     0     0     0     0     0     0   
Assorted Desserts                     0     0     0     0    25     0     0   

                                         
CustomerID                        98200  
ProductName                              
Anchovy Paste - 56 G Tube             0  
Appetizer - Mini Egg Roll, Shrimp     0  
Appetizer - Mushroom Tart             0  
Appetizer - Sausage Rolls             0  
Apricots - Dried                      0  
Apricots - Halves                    25  
Apricots Fresh                       25  
Arizona - Green Tea                   0  
Artichokes - Jerusalem                0  
Assorted Desserts                    25  

[10 rows x 1000 columns]

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
squareform(pdist(pivot.T, 'euclidean'))

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [74]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'euclidean'))), 
                         index=pivot.columns, columns=pivot.columns)

distances.head()



distances['Quantity'].columns




distances['Quantity',33].head()


#distances['Quantity'][33]



          CustomerID
Quantity  33            1.000000
          200           0.077421
          264           0.087047
          356           0.081800
          412           0.080634
Name: (Quantity, 33), dtype: float64

In [71]:
#distances.reset_index(drop=True, inplace=True)


#distances.columns = distances.columns.droplevel()

#.droplevel()

In [76]:
distances.head()

Quantity                                          \
CustomerID              33        200       264       356       412     
         CustomerID                                                     
Quantity 33          1.000000  0.077421  0.087047  0.081800  0.080634   
         200         0.077421  1.000000  0.078448  0.076435  0.073693   
         264         0.087047  0.078448  1.000000  0.080070  0.081800   
         356         0.081800  0.076435  0.080070  1.000000  0.076435   
         412         0.080634  0.073693  0.081800  0.076435  1.000000   

                                                                       ...  \
CustomerID              464       477       639       649       669    ...   
         CustomerID                                                    ...   
Quantity 33          0.082709  0.074573  0.083020  0.081503  0.080070  ...   
         200         0.075255  0.075956  0.076435  0.077674  0.076923  ...   
         264         0.080350  0.076923  0.080634  0.082100  0.078448  ...   
         356         0.078187  0.075025  0.082403  0.077171  0.075956  ...   
         412         0.078711  0.075025  0.082403  0.078187  0.078448  ...   

                                                                       \
CustomerID              97697     97753     97769     97793     97900   
         CustomerID                                                     
Quantity 33          0.004811  0.004669  0.004412  0.005019  0.004312   
         200         0.004824  0.004681  0.004431  0.005047  0.004311   
         264         0.004822  0.004674  0.004416  0.005035  0.004322   
         356         0.004816  0.004671  0.004416  0.005038  0.004310   
         412         0.004810  0.004702  0.004414  0.005034  0.004318   

                                                                       
CustomerID              97928     98069     98159     98185     98200  
         CustomerID                                                    
Quantity 33          0.004515  0.004583  0.004355  0.004167  0.004333  
         200         0.004521  0.004614  0.004367  0.004166  0.004335  
         264         0.004543  0.004595  0.004365  0.004179  0.004333  
         356         0.004526  0.004578  0.004365  0.004175  0.004339  
         412         0.004530  0.004578  0.004367  0.004177  0.004349  

[5 rows x 1000 columns]

In [80]:
similarities = distances['Quantity',200].sort_values(ascending=False)[1:6]
similarities

          CustomerID
Quantity  3317          0.084959
          1072          0.083972
          3535          0.082403
          1920          0.082100
          3909          0.080920
Name: (Quantity, 200), dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [106]:
similarCustomer = df.copy()
similarCustomer.reset_index(inplace=True)

similarCustomer.head()

#similarCustomer[similarCustomer['CustomerID']==3317].head()



,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


In [103]:


appended_data = []
for name, score in dict(similarities).items():
    appended_data.append(similarCustomer[similarCustomer['CustomerID']==name[1]])

Similar = pd.concat(appended_data)
    
Similar.head()

,CustomerID,ProductName,Quantity
2788,3317,Bay Leaf,1
2789,3317,Beef - Ground Medium,1
2790,3317,Beef - Rib Eye Aaa,1
2791,3317,Butter - Unsalted,1
2792,3317,Cheese - Victor Et Berthold,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [111]:
Purchased = pd.DataFrame(Similar['Quantity'].groupby([data['ProductName']]).sum().sort_values(ascending=False))
Purchased.index
Purchased

,Quantity
ProductName,
French Pastry - Mini Chocolate,55
Skirt - 29 Foot,53
"Wine - White, Mosel Gold",50
Sponge Cake Mix - Chocolate,46
Lettuce - Frisee,34
Cheese - Parmesan Cubes,28
Raspberries - Fresh,10


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [172]:
purchased = pd.DataFrame(Purchased.index)
#purchased['CustomerID'] = 200
#purchased['Quantity'] = 0

purchased

,ProductName
0,French Pastry - Mini Chocolate
1,Skirt - 29 Foot
2,"Wine - White, Mosel Gold"
3,Sponge Cake Mix - Chocolate
4,Lettuce - Frisee
5,Cheese - Parmesan Cubes
6,Raspberries - Fresh


In [128]:
df200 = similarCustomer[similarCustomer['CustomerID']==200]
df200.head()

,CustomerID,ProductName,Quantity
59,200,Bacardi Breezer - Tropical,1
60,200,Bagel - Plain,1
61,200,Bar - Granola Trail Mix Fruit Nut,1
62,200,Beef - Ground Medium,1
63,200,Beef - Montreal Smoked Brisket,1


In [192]:
newdf200 = pd.merge(df200,purchased, on='ProductName', how='outer')
newdf200.fillna(0).sort_values(by='Quantity', ascending=True).head()['ProductName'].tolist()

['Juice - Happy Planet',
 'Wine - White, Mosel Gold',
 'French Pastry - Mini Chocolate',
 'Skirt - 29 Foot',
 'Beef - Rib Eye Aaa']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [187]:
recommendations = {}

In [186]:
listCustomerID = data['CustomerID'].unique().tolist()

In [193]:

for customer in listCustomerID:
    
    similarities = distances['Quantity', customer].sort_values(ascending=False)[1:6]

    appended_data = []
    for name, score in dict(similarities).items():
        appended_data.append(similarCustomer[similarCustomer['CustomerID']==name[1]])
    Similar = pd.concat(appended_data)

    Purchased = pd.DataFrame(Similar['Quantity'].groupby([data['ProductName']]).sum().sort_values(ascending=False))
    purchased = pd.DataFrame(Purchased.index)

    df200 = similarCustomer[similarCustomer['CustomerID']== customer]
    newdf200 = pd.merge(df200,purchased, on='ProductName', how='outer')
    recommendations[customer] = newdf200.fillna(0).sort_values(by='Quantity', ascending=True).head()['ProductName'].tolist()
    


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [196]:
result = pd.DataFrame(recommendations)
result.head()

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,25270,17390,97029,38154,74403,49005,41286,85878,68506,3472
0,Bananas,General Purpose Trigger,Sunflower Seed Raw,Longos - Grilled Salmon With Bbq,Cookie Dough - Double,General Purpose Trigger,"Wine - White, Colubia Cresh",Pail For Lid 1537,Longos - Grilled Salmon With Bbq,Cheese - Parmesan Cubes,...,Longos - Grilled Salmon With Bbq,Cheese - Parmesan Cubes,Sherry - Dry,"Rosemary - Primerba, Paste",Sherry - Dry,Milk - 2%,Pail For Lid 1537,Beef - Texas Style Burger,Sunflower Seed Raw,Juice - Happy Planet
1,Nantucket - Pomegranate Pear,"Garlic - Primerba, Paste","Cup - 6oz, Foam",Cheese Cloth No 100,Rum - Mount Gay Eclipes,Cookie Dough - Double,Cheese Cloth No 100,Wine - Vineland Estate Semi - Dry,"Rosemary - Primerba, Paste","Lamb - Pieces, Diced",...,Beer - Rickards Red,French Pastry - Mini Chocolate,"Mushroom - Trumpet, Dry",Pail For Lid 1537,Cod - Black Whole Fillet,Liners - Baking Cups,"Soup - Campbells, Beef Barley",Quiche Assorted,Knife Plastic - White,"Wine - White, Mosel Gold"
2,Pail For Lid 1537,Milk - 2%,Cornflakes,Arizona - Green Tea,Sausage - Breakfast,Campari,Beef - Inside Round,Fond - Neutral,Beef - Inside Round,French Pastry - Mini Chocolate,...,Muffin - Carrot Individual Wrap,Juice - Happy Planet,Tomatoes Tear Drop,"Soup - Canadian Pea, Dry Mix",Island Oasis - Mango Daiquiri,"Soup - Campbells, Beef Barley",Fond - Neutral,Pastry - Butterscotch Baked,"Cup - 6oz, Foam",French Pastry - Mini Chocolate
3,Pastry - Raisin Muffin - Mini,Pork - Hock And Feet Attached,Wine - Toasted Head,Beef - Inside Round,Island Oasis - Mango Daiquiri,Pork - Inside,"Rosemary - Primerba, Paste",Muffin - Carrot Individual Wrap,Wine - Ruffino Chianti,"Wine - White, Colubia Cresh",...,Wine - Fume Blanc Fetzer,"Rosemary - Primerba, Paste",Bacardi Breezer - Tropical,Sunflower Seed Raw,"Pepsi - Diet, 355 Ml",Vol Au Vents,Vol Au Vents,Tray - 16in Rnd Blk,Island Oasis - Mango Daiquiri,Skirt - 29 Foot
4,"Peas - Pigeon, Dry",Loquat,Cocktail Napkin Blue,Eggplant - Asian,Wine - Toasted Head,Fondant - Icing,Skirt - 29 Foot,Nantucket - Pomegranate Pear,Beer - Original Organic Lager,"Wine - White, Mosel Gold",...,"Wine - White, Mosel Gold","Wine - White, Colubia Cresh",Pasta - Orecchiette,Phyllo Dough,Pastry - Choclate Baked,Wine - Fume Blanc Fetzer,Nantucket - Pomegranate Pear,Sauce - Demi Glace,Cake - Cake Sheet Macaroon,Beef - Rib Eye Aaa


In [202]:
result2 = result.T
result2.columns = ['Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
result2['CustomerID'] = result2.index
new_order = [5, 0, 1, 2, 3, 4]
result2 = result2[result2.columns[new_order]]
result2.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
61288,61288,"Turnip - White, Organic","Beer - Alexander Kieths, Pale Ale",Otomegusa Dashi Konbu,Bananas,Wine - Magnotta - Cab Sauv
77352,77352,Vanilla Beans,Smoked Paprika,Cookies - Assorted,"Oranges - Navel, 72",Dried Figs
40094,40094,Dc Hikiage Hira Huba,Scallop - St. Jaques,Soup Knorr Chili With Beans,Wine - Toasted Head,Crush - Cream Soda
23548,23548,Pop Shoppe Cream Soda,"Rum - Coconut, Malibu","Soup - Campbells, Lentil","Coconut - Shredded, Sweet",Grapes - Red
78981,78981,"Tart Shells - Sweet, 4",Sobe - Tropical Energy,Scallops - Live In Shell,Wine - Magnotta - Cab Sauv,"Salmon - Atlantic, Skin On"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [197]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'correlation'))), 
                         index=pivot.columns, columns=pivot.columns)

for customer in listCustomerID:
    
    similarities = distances['Quantity', customer].sort_values(ascending=False)[1:6]

    appended_data = []
    for name, score in dict(similarities).items():
        appended_data.append(similarCustomer[similarCustomer['CustomerID']==name[1]])
    Similar = pd.concat(appended_data)

    Purchased = pd.DataFrame(Similar['Quantity'].groupby([data['ProductName']]).sum().sort_values(ascending=False))
    purchased = pd.DataFrame(Purchased.index)

    df200 = similarCustomer[similarCustomer['CustomerID']== customer]
    newdf200 = pd.merge(df200,purchased, on='ProductName', how='outer')
    recommendations[customer] = newdf200.fillna(0).sort_values(by='Quantity', ascending=True).head()['ProductName'].tolist()



In [203]:
result = pd.DataFrame(recommendations)

result2 = result.T
result2.columns = ['Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
result2['CustomerID'] = result2.index
new_order = [5, 0, 1, 2, 3, 4]
result2 = result2[result2.columns[new_order]]
result2.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
61288,61288,"Turnip - White, Organic","Beer - Alexander Kieths, Pale Ale",Otomegusa Dashi Konbu,Bananas,Wine - Magnotta - Cab Sauv
77352,77352,Vanilla Beans,Smoked Paprika,Cookies - Assorted,"Oranges - Navel, 72",Dried Figs
40094,40094,Dc Hikiage Hira Huba,Scallop - St. Jaques,Soup Knorr Chili With Beans,Wine - Toasted Head,Crush - Cream Soda
23548,23548,Pop Shoppe Cream Soda,"Rum - Coconut, Malibu","Soup - Campbells, Lentil","Coconut - Shredded, Sweet",Grapes - Red
78981,78981,"Tart Shells - Sweet, 4",Sobe - Tropical Energy,Scallops - Live In Shell,Wine - Magnotta - Cab Sauv,"Salmon - Atlantic, Skin On"
